In [1]:
from DistMLIP.implementations.mace import MACECalculator_Dist
from mace.calculators import mace_mp

import matgl
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor

import os

/data/kevinxhan/miniconda3/envs/distmlip/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


In [2]:
# Insert your atoms
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(3.5), ["Li", "Mn"], [[0, 0, 0], [0.5, 0.5, 0.5]])
struct.perturb(0.5)
struct.make_supercell((30, 30, 30))
print(f"There are {len(struct)} atoms.")
atoms = AseAtomsAdaptor().get_atoms(struct)

There are 54000 atoms.


In [3]:
# Load/create a previously trained MACECalculator on cpu
from mace.calculators import mace_mp

calc = mace_mp(model="small", device="cpu")

# Make a distributed version of the calculator 
dist_calc = MACECalculator_Dist.from_existing(calc)
# Enable distributed mode and specify the GPUs to use
dist_calc.enable_distributed_mode([0, 1])


Using Materials Project MACE for MACECalculator with /home/kevinxhan/.cache/mace/20231210mace128L0_energy_epoch249model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.


/data/kevinxhan/miniconda3/envs/distmlip/lib/python3.10/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


In [5]:
# You can set the number of threads to use via environment variable
os.environ["DISTMLIP_NUM_THREADS"] = "8"

# Performing a static calculation on the ASE atoms object
dist_calc.calculate(atoms)

# Retrieiving results
dist_calc.results['energy'], dist_calc.results['forces']

Failed to parse input arguments. Inputs should be of the following format: double[:, ::1] all_coords, double r, long[::1] pbc, const double[:, ::1] lattice, int num_nodes, unsigned int num_partitions, double bond_r=3.0, double tol=1e-8, int number_of_threads=256, bint use_bond_graph=True, frac_coords


TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
dist_calc.results['stress']

In [ ]:
# You can use MACECalculator_Dist the same way as MACECalculator. An example using DistMLIP's MolecularDynamics class from MatGL:
from DistMLIP.implementations.matgl import MolecularDynamics

driver = MolecularDynamics(
    atoms,
    potential=dist_calc,
    timestep=0.5,
    temperature=300,
    loginterval=200
)

driver.run(20)